## LLM App/Project - RAG testing / Text Classification

### Steps: 

1. Create a chat classifier in lamaindex which based on the provided BBC headline classify it into one of the five news categories.
2. Using the chat classifier from the first task, add output evaluation - consider a possiblity of adding the way of teaching/improving a chat output based on this information
3. Write RAG - add some wiki documents (~ 20 documents) and evaluate the responses
4. Experiment with different evaluation metrics
5. Experiment with different RAG methodologies and use evaluation metrics to see different results they can provide

Possible problems:
- Gemini API restriction

In [38]:
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

llm = Gemini(
    model="models/gemini-1.5-flash",
    api_key=GOOGLE_API_KEY  # uses GOOGLE_API_KEY env var by default
)

In [39]:
messages = [
    ChatMessage(
        role="system",
        content=(
            "You are a text classifier specializing in BBC headlines. "
            "I will provide you with a headline, and you will classify it into one of the following five categories: "
            "business, entertainment, politics, sport, or tech. "
            "Choose the category that best fits the headline. If a headline fits multiple categories, select the one most relevant. "
            "Provide only the category name as the response, without any additional text."
        )
    )
]

resp = llm.chat(messages)
print(f"System Response: {resp}")

# Chat loop 
while True:
    text_input = input("User: ")
    if text_input.lower() == "exit":
        print("Exiting classifier. Goodbye!")
        break
    
    messages.append(ChatMessage(role="user", content=text_input))

    response = str(llm.chat(messages))
    messages.append(ChatMessage(role='assistant', content=response))    
    print(f"\nChat: {response}\n")

System Response: assistant: Okay, I'm ready. Please provide the headline.

